<a href="https://colab.research.google.com/github/kvncp/advent_of_code/blob/main/AoC_Day_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
"""
0--->1
^    |
|    |
|    v
3<---2

[01, 12, 23, 30]

Rotate:
3--->0
^    |
|    |
|    v
2<---1

[30, 01, 12, 23]

Flip:
1----0
|    |
|    |
|    |
2----3

[10, 03, 32, 21]
"""

import collections
import functools
import itertools

TOP = 0
RIGHT = 1
BOTTOM = 2
LEFT = 3

def parse_edge(edge):
  # return edge
  return int(edge.replace('#', '1').replace('.', '0'), 2)

def flip_edge(n, no_of_bits=10):
  result = 0
  for i in range(no_of_bits):
      result <<= 1
      result |= n & 1
      n >>= 1
  return result

def flip_tile(tile):
  t = [flip_edge(t) for t in tile]
  t[RIGHT], t[LEFT] = t[LEFT], t[RIGHT]
  return t

def rotate_tile(tile):
  return [tile[-1]] + tile[:-1]

def flip_image(image):
  return [[x for x in row[::-1]] for row in image]

def rotate_image(image):
  return list(zip(*image[::-1]))

def strip_edges(image):
  return [[c for c in row[1:-1]] for row in image[1:-1]]

def parse_tiles(tiles):
  tile_map = {}
  edge_map = {}
  for t in tiles:
    t = t.split('\n')
    id = int(t[0].split()[1].replace(':', ''))
    edge_map[id] = []
    tile_map[id] = t[1:]

    # Tile edges are defined clockwise.
    edge_map[id].append(parse_edge(t[1]))
    edge_map[id].append(parse_edge(''.join(x[-1] for x in t[1:])))
    edge_map[id].append(parse_edge(t[-1][::-1]))
    edge_map[id].append(parse_edge(''.join(x[0] for x in reversed(t[1:]))))
  return tile_map, edge_map

MONSTER = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """
def parse_monster(monster):
  coordinates = []
  for i, row in enumerate(monster.split('\n')):
    for j, c in enumerate(row):
      if c == '#':
        coordinates.append((i, j))
  return coordinates

parse_tiles(tiles)

({1171: ['####...##.',
   '#..##.#..#',
   '##.#..#.#.',
   '.###.####.',
   '..###.####',
   '.##....##.',
   '.#...####.',
   '#.##.####.',
   '####..#...',
   '.....##...'],
  1427: ['###.##.#..',
   '.#..#.##..',
   '.#.##.#..#',
   '#.#.#.##.#',
   '....#...##',
   '...##..##.',
   '...#.#####',
   '.#.####.#.',
   '..#..###.#',
   '..##.#..#.'],
  1489: ['##.#.#....',
   '..##...#..',
   '.##..##...',
   '..#...#...',
   '#####...#.',
   '#..#.#.#.#',
   '...#.#.#..',
   '##.#...##.',
   '..##.##.##',
   '###.##.#..'],
  1951: ['#.##...##.',
   '#.####...#',
   '.....#..##',
   '#...######',
   '.##.#....#',
   '.###.#####',
   '###.##.##.',
   '.###....#.',
   '..#.#..#.#',
   '#...##.#..'],
  2311: ['..##.#..#.',
   '##..#.....',
   '#...##..#.',
   '####.#...#',
   '##.##.###.',
   '##...#.###',
   '.#.#.#..##',
   '..#....#..',
   '###...#.#.',
   '..###..###'],
  2473: ['#....####.',
   '#..#.##...',
   '#.##..#...',
   '######.#.#',
   '.#...#.#.#',
   '.#########',
   '.##

In [61]:
class Image(object):
  def __init__(self, tiles):
    self.tiles = tiles
    self.tile_to_image, self.tile_to_edges = parse_tiles(tiles)
    self.edge_to_tiles = collections.defaultdict(set)
    for k, v in self.tile_to_edges.items():
      for e, i in enumerate(v):
        self.edge_to_tiles[i].add((k, e, 1))
        self.edge_to_tiles[flip_edge(i)].add((k, e, -1))
    self.num_tiles = len(self.tile_to_edges)
    self.edge_len = int(self.num_tiles**0.5)

  def find_corners(self):
    outside_edges = {k:v for k, v in self.edge_to_tiles.items() if len(v) == 1}

    edge_map = collections.defaultdict(set)
    for ((tile_id, edge, orientation),) in outside_edges.values():
      edge_map[tile_id].add(edge)

    corners = {k for k, v in edge_map.items() if len(v) == 2}
    edges = {k for k, v in edge_map.items() if len(v) == 1}

    return corners

  def arrange_tiles(self):
    self.image = [0]*(self.num_tiles)
    corners = self.find_corners()

    # Remember operations. This will help reconstruct the image later.
    self.operations = {}
    index = 0
    # Make this deterministic, so always place the corner with the largest ID
    # in the upper left slot.
    top_left_tile = max(corners)
    self.image[index] = top_left_tile
    self.orient_tile(top_left_tile, None, None)

    for i in range(1, self.num_tiles):
      tile_id, edge, position = self.find_tile(i)
      self.orient_tile(tile_id, edge, position)
      self.image[i] = tile_id

  def print_image(self, image):
    print('\n'.join(''.join(x) for x in image))

  def find_tile(self, index):
    tile_index = self._get_left_index(index)
    adjacent_edge = RIGHT
    if tile_index is None:
      tile_index = self._get_top_index(index)
      adjacent_edge = BOTTOM
    
    tile_id = self.image[tile_index]
    edge = self.tile_to_edges[tile_id][adjacent_edge]
    for t in self.edge_to_tiles[edge]:
      if t[0] != tile_id:
        return t[0], flip_edge(edge), (adjacent_edge - 2) % 4

  def orient_tile(self, tile_id, edge, position):
    # Special case the top left tile:
    if edge is None and position is None:
      edges, flip, rot = self._top_left_corner(
          self.tile_to_edges[tile_id])
    else:
      edges = self.tile_to_edges[tile_id]
      if edge not in edges:
        edges = flip_tile(edges)
        flip = True
      else:
        flip = False

      rot = 0
      while edges[position] != edge:
        edges = rotate_tile(edges)
        rot += 1

    self.tile_to_edges[tile_id] = edges
    self.operations[tile_id] = (flip, rot)

  def _top_left_corner(self, tile):
    flip, rotations = False, 0
    while len(self.edge_to_tiles[tile[TOP]]) != 1 or len(
        self.edge_to_tiles[tile[LEFT]]) != 1:
      tile = rotate_tile(tile)
      rotations += 1

    return tile, flip, rotations

  def _get_left_index(self, index):
    i = (index % self.edge_len) - 1
    if i < 0:
      return None
    return index - 1

  def _get_top_index(self, index):
    i = index - self.edge_len 
    if i < 0:
      return None
    return index - self.edge_len

  def construct_image(self):
    arranged_images = []
    for tile_id in self.image:
      flip, rot = self.operations[tile_id]
      image = strip_edges(self.tile_to_image[tile_id])
      if flip:
        image = flip_image(image)
      for r in range(rot):
        image = rotate_image(image)
      
      arranged_images.append(image)

    full_image = []
    for row_index in range(self.edge_len):
      start = row_index*self.edge_len
      end = start + self.edge_len
      for i in zip(*arranged_images[start:end]):
        full_image.append(''.join(''.join(x) for x in i))

    return full_image
  
  def find_sea_monster(self, image):
    monster = parse_monster(MONSTER)

    for flip in range(2):
      for rot in range(3):
        monsters = self._search_for_monster(image, monster)
        if monsters:
          return monsters
        image = rotate_image(image)
      image = flip_image(image)    
  
  def _search_for_monster(self, image, monster):
    height = max(x[0] for x in monster)
    width = max(x[1] for x in monster)

    found_monsters = 0

    for row_index in range(len(image)-height):
      for col_index in range(len(image[row_index])-width):
        for r, c in monster:
          if image[r+row_index][c+col_index] != '#':
            break
        else:
          found_monsters += 1
    
    return found_monsters

In [64]:
image = Image(tiles)
corners = image.find_corners()
print(functools.reduce(lambda x, y: x*y, corners))
image.arrange_tiles()
print(image.image)
a = image.construct_image()

num_monsters = image.find_sea_monster(rotate_image(a))

full_image = '\n'.join(''.join(x) for x in a)
full_image.count('#') - MONSTER.count('#')*num_monsters

79412832860579
[3833, 3943, 2917, 2791, 1693, 1051, 1367, 2851, 2543, 3413, 1429, 3001, 1013, 1307, 2393, 1151, 2719, 3119, 2063, 3877, 1163, 2677, 1063, 1279, 1213, 2281, 1109, 2521, 2659, 1103, 2633, 3947, 1231, 2143, 2549, 2111, 3881, 2707, 2447, 2963, 2531, 2339, 3583, 2221, 3547, 1039, 2417, 3109, 3301, 3169, 3529, 3217, 2939, 2351, 3019, 2053, 2027, 2089, 1291, 2099, 2671, 3769, 3257, 2477, 1097, 3079, 1979, 1181, 2909, 1579, 2269, 2399, 1811, 1559, 3617, 2081, 2683, 3359, 2161, 2137, 1511, 2239, 1409, 2003, 3643, 3559, 1889, 1999, 3517, 3691, 1709, 2371, 1223, 2857, 2557, 1997, 2411, 3853, 1423, 1949, 1471, 1009, 2207, 1381, 1723, 3701, 2957, 2423, 3613, 1667, 2273, 2579, 1873, 3607, 1289, 2693, 1847, 1789, 1567, 2287, 2837, 3307, 1087, 3821, 2833, 1091, 1901, 1447, 1753, 2731, 1583, 3793, 3433, 1973, 3319, 2749, 2801, 3083, 1451, 2843, 1867, 3767, 3491, 2011]


2155

In [1]:
tiles = """Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...""".split('\n\n')

In [63]:
tiles = """Tile 1579:
.#.##.#..#
#..#......
.#....#...
#.....#...
#..#....##
#.##.....#
#....#....
......###.
..#..#.###
......##.#

Tile 3413:
..##.#....
..##...#..
#....#.#.#
#..#.##...
.###...#.#
#..#..##.#
..#......#
.......#.#
#.....#.##
..#...####

Tile 3559:
#...###..#
##..#.....
.#.#.#....
####....#.
.####....#
..#.#.....
##........
...####.#.
#.......##
...#.#...#

Tile 3947:
#...#..#..
#....##...
.......#..
##.......#
###..##.##
#...#..###
....######
.#....##.#
..........
#....####.

Tile 1063:
#..###..#.
##.#...##.
...##..#.#
#.....####
.#...#....
#...##.#..
#..#....##
...##...#.
#.....#.##
#..#...#..

Tile 3613:
.##.#####.
..#.......
..........
..#.......
.#..#.....
.#.....#..
..###....#
........#.
#....#...#
###.....#.

Tile 3019:
#...#.####
#..#..#..#
..........
......###.
.#.#.....#
#.....#.##
....#.#.#.
##.......#
#..#......
##...###..

Tile 1429:
.#######..
####.....#
.#.##.#...
...##.#..#
....#.###.
#..##.....
...#..####
..#..##..#
#.##......
..##.#..#.

Tile 2543:
....##..#.
#...#.....
#..#..#..#
...#.###..
......####
...#...###
......####
#...#.#..#
..#.....##
...#.##.##

Tile 2447:
.#..#####.
##.#.....#
.....#...#
..#..#..##
#.#...#..#
##.#..#...
#####.#...
##.......#
...#..####
..###.###.

Tile 1471:
.##.###.#.
#...#...##
......##..
.....#....
#....#....
#.....#...
#...#.#...
.......#..
##..#....#
...#..#.##

Tile 1753:
#....#####
.######.#.
.#....##.#
#.........
......#..#
#...##....
#........#
###.......
#..#..#...
..#..#....

Tile 3583:
.##..#####
.........#
##.#...##.
..........
####..##.#
#..#.....#
.#...#...#
...#..###.
....##..#.
..#.##..##

Tile 1447:
###..###..
..#..#.#..
.#....#..#
#..#......
...###...#
#....#....
#....##...
#.####...#
##..###.##
.##...###.

Tile 2143:
##.##.#.#.
....#.....
..........
.......#.#
.#........
#..#....#.
...#......
#..#..#..#
..#.#.#..#
###....#.#

Tile 3433:
##.#...###
......####
.#........
#.##....##
##..#....#
.#...###.#
##.......#
##.....##.
.........#
.#........

Tile 2671:
#...#..###
#...#.#.#.
#...#.#..#
#.#....#.#
#..##.#.##
...#######
#.#..#..#.
#.#.....##
.......#..
.#....##..

Tile 3793:
.#.#.##.#.
##.......#
###..#....
#....#.#..
#..##.#...
#.....##..
.#...##..#
........#.
.##.....#.
###.######

Tile 1109:
....#.###.
.#.......#
#...#....#
..#.#.#.#.
....#.#..#
#.#.......
#.#.....##
.........#
##..#..#..
..####..#.

Tile 2837:
.#.#......
#.........
#.#.......
##.....#..
#...#....#
.....#...#
....###...
###.......
##.....#..
###...#.##

Tile 3529:
.###.#.##.
..#.#....#
.#..##...#
...#...#..
..##...#..
.....#.#.#
....#....#
#...#....#
.#..#..###
..##...##.

Tile 2161:
##..##..#.
.#..#.####
.....#...#
#....#...#
#.#...#...
#.....####
##..##....
....#.#..#
.....#....
.#####.#.#

Tile 2939:
#.........
......#..#
.....##...
....##.#.#
..##.....#
#...#####.
#...###...
#........#
.#......##
...###....

Tile 1409:
.##...####
.....#...#
#......##.
#.#.##..#.
#..#...###
.##......#
#.##...#.#
#.#....#..
.#.....#..
.#..#.....

Tile 2099:
..##.####.
#..##....#
#......#.#
.....#...#
#..#..#...
##.##....#
....#.....
#.....#..#
#...##....
#.##.#..#.

Tile 2081:
###.#.#.#.
.#.##.#..#
###..#####
#.#....#..
...#......
#...##...#
#.#....###
.##...#...
.##...##.#
#.#..##..#

Tile 3943:
..#..##.##
#....#.##.
..##...###
.#.#...#.#
#..####...
#..#...#.#
.##.#..#..
###.....#.
..#.....#.
####.#####

Tile 1867:
.##.#...#.
#..#...##.
#.#.#.....
#...#.....
.........#
..###...#.
...#......
.###..#..#
....#..##.
#...###.#.

Tile 1367:
#.#.#.....
..#.#..#..
........#.
..........
...#.....#
..#.##.##.
..#.#....#
.....#.##.
#......#..
##...####.

Tile 2207:
#.#..#.##.
##..#.#..#
...####...
.#.#...###
.......#..
#...#....#
#.##..#...
....#...##
.........#
.###.#..#.

Tile 2221:
....##.#..
.##....##.
.#..###.##
.#........
#.#.##.#..
....###..#
##.#..##..
..#....#..
##..#..#..
##.#.##..#

Tile 2053:
##.#.#....
#.........
.........#
#..#.#..##
#..#.....#
#....#.#..
#......#..
.....#....
.#.###...#
..##..#.##

Tile 3547:
####.####.
....#.#.##
##.#.#..##
##......#.
#.....#..#
..........
#.#.....##
.#........
.#.#..##..
##.#.##..#

Tile 1307:
#.##.#...#
......#..#
#....#....
#.......##
#.....#...
...##..#.#
.#.#.##...
....##..#.
...#...#.#
..#..#..##

Tile 2683:
#..##..#.#
..#.....##
###.##.#..
##....#...
#.##..#..#
######...#
..##..#..#
####.....#
....####..
#.#.##.#..

Tile 1973:
..##.##..#
#.#.#....#
#..#.#..#.
#...##....
.......#..
.......##.
#...#...##
..##....#.
#...#...##
.#####..##

Tile 2857:
..###.##.#
#..###....
.#.###..#.
#........#
...#.....#
##...#....
.##.###...
#....#...#
.....#...#
#.##...#..

Tile 3079:
##.....###
....#...##
#...##....
.#.#.....#
#.###.##..
###..#.###
#........#
...#......
#........#
#..###.##.

Tile 2281:
###...####
#..#.....#
#.....#...
#.#..#....
##.#...#.#
....##....
...#.#....
##.......#
..#..#....
..##.#.##.

Tile 2677:
####...#..
........##
.....##...
........#.
...#..#..#
...#...#.#
##.#.....#
.##..#.#..
..........
#.#....###

Tile 1151:
....#...##
..#..#..##
##.......#
..#..###..
..#...#..#
...#....#.
#....#....
.#....##.#
##.#.#####
#...##..#.

Tile 1979:
.###.#.#.#
..#..#....
.....#...#
#.#...#.#.
#....##..#
.#..####.#
.........#
##......#.
..#.....##
###.#...##

Tile 3853:
.#.#..#.##
####...#.#
....##...#
..##...#..
.......#.#
..##.#...#
#..#.#...#
.....##.#.
......#...
..#.##..#.

Tile 3359:
.#####.#.#
#..#.#...#
##.#......
...#....#.
....#.#...
..#..###..
#....####.
.......#.#
.#.......#
..#.##.#.#

Tile 3701:
.#.##.##.#
#..#......
#...#..#..
..#.....#.
#...#.....
.##.#....#
#...#..##.
#.##.....#
#...#.....
#.##...#..

Tile 2063:
##.......#
..........
#.#..#....
...##.##..
..#.###..#
#.........
......#..#
.#......#.
.##..####.
......####

Tile 1289:
.####.....
#.....#.#.
#.........
.#....#..#
#..#.....#
.#..#...#.
...#..#..#
#...##...#
...#....##
##.#..###.

Tile 1873:
..#.###.##
##.#.#....
....#..#..
.#.##.....
.#.#......
.####.#.#.
.#......#.
...##..#.#
#...#.#..#
###.##..#.

Tile 1559:
..####.#.#
#.#.#....#
....#.##..
#.##..###.
.#.###....
##....#..#
...#...#..
.#.#......
.#..#.....
...##.####

Tile 2287:
.###..####
#...#...#.
......###.
##.#......
#.#....#.#
.....#.#.#
..##.#...#
.#..#.##.#
##...#.###
.######...

Tile 2423:
.#.##...#.
#......#..
.#.#.##...
###.#...##
#.##..#...
.###.....#
##.###.#.#
#........#
##...#..##
.##.#..##.

Tile 2269:
..#.....##
...##.#..#
#.........
....##....
##.......#
......#..#
.......#.#
##...#....
...#....#.
...####.#.

Tile 1223:
...##...#.
#..###....
#..#..#..#
.#........
....##....
#.#......#
#........#
.....#....
.###....#.
##....####

Tile 1723:
.##..#..##
#..#.....#
.#....#..#
.##.......
...#..#..#
##........
##....##..
........##
....##....
..#.#....#

Tile 2371:
#...#.##..
#......##.
.#.#...#..
##...#..##
#...#....#
#..#.#.#..
#.#.....##
.....#..##
.##..###..
#..##..#..

Tile 2339:
#..###..##
..#..#.#..
##.......#
#..#......
#....##...
#..#...##.
.....#.#.#
#.#....#.#
#.#..#..#.
.#.##....#

Tile 1693:
.##.......
..#.###.#.
##.......#
##....#..#
#...#.....
#.#......#
..#.....#.
#.#......#
#.......#.
.#....###.

Tile 1811:
.#..#..#..
.#....##..
#.#...#..#
#....#...#
#...#.....
#.##..##..
#..##.....
#.....#...
#..#.....#
####.##...

Tile 2137:
###..#..#.
....#.#.##
.#..#..#.#
##....#.##
#....##...
##...#.##.
#.##..#...
.#........
##......#.
##..##..#.

Tile 2693:
.###..#.##
....#.#.#.
#.#.......
#...#.#...
#....#....
....##...#
.....#..#.
#.##...##.
#.#....###
#..#..##.#

Tile 3833:
##.##.....
#....#...#
#.......#.
#.......#.
##.##.....
..##.....#
....#...##
##.......#
..#.......
#####.####

Tile 2351:
#..###.##.
.........#
..#...#..#
..#.###...
#.#.#.#...
..##.....#
#.##..#..#
#.....#.#.
##.#....##
#.####.##.

Tile 3607:
.#.###....
.#..#....#
###...##.#
...#....##
#...###..#
#..###....
#..#....#.
...#.#....
##.#..#...
##.###..#.

Tile 1097:
.#.##.#.##
.#..##.#..
.......#.#
..........
#.#.#...##
##.....#.#
#...##...#
.#.......#
.##.##..#.
...##....#

Tile 3217:
..#.###...
.#..#....#
#..#....##
#...#...##
#.###...#.
.####..#.#
..#...#...
..#.#....#
..##.#...#
#.......#.

Tile 3083:
###..##.##
#........#
#.#.#.#..#
.....##...
..#..###..
#.###...##
........##
##...#....
##...#....
#..##.#..#

Tile 2273:
..###..#..
##.##.#.#.
......##..
......##..
.##..#..##
#..#....#.
.#.##.#..#
...##..###
#.....#..#
##..####.#

Tile 2957:
.####.#...
#.........
...#......
...#..#..#
..##......
#........#
..#.#.#..#
.##.......
...#.....#
####.#.##.

Tile 2521:
###.####.#
###..#..#.
#.........
...#...###
#..#...#..
.....##...
.....##.##
##........
#....#.#.#
..#..##.#.

Tile 1039:
##.#..####
#.......#.
.#.......#
#......##.
#.#...#...
..#..#.#..
###....###
.#..#....#
#....###.#
.####.####

Tile 2003:
....####..
.#...##..#
...#.....#
....#.....
#.......#.
..#....#..
....#.#.##
.#..##...#
...#...#.#
..###.####

Tile 3307:
##..#.#.#.
##.#......
....#.#...
.##.....#.
.........#
....##..##
..##......
##........
.....###..
##.#....##

Tile 2557:
...##.#...
.#....#.##
##....##..
#......#..
#......#..
......#..#
#.#...#...
#...#.#...
..........
.#.#.#.#.#

Tile 2731:
..#.###.#.
#.#..#....
#..#..#..#
#####..#..
#.....####
.....#.#..
........##
#.#.##...#
...#......
...#.#.#.#

Tile 2011:
#...####.#
#.....#.#.
###..#.#..
#...#...##
#..##.##..
....#.....
#.....#.#.
.#.....#..
##...#...#
..###..##.

Tile 2801:
#..#.##...
###.......
###.#.....
......#...
...##.....
#.........
..##..###.
##.#....##
#...##..#.
##.##..#.#

Tile 2909:
#..##...#.
........#.
....#.#...
.#....#.#.
#.......##
##..##....
....#..#..
..#....#..
#.#.....##
#.###.##..

Tile 2549:
#.#.#....#
#.###.....
.....#....
###.#.....
........#.
#.........
#..#...#.#
.#...#....
#......#..
######.##.

Tile 1567:
...#.##.#.
#.#.#..#..
##.#..###.
#.......##
##........
#..#.#..#.
#.........
....#..#..
..........
.#.#####..

Tile 1231:
.##.#.#..#
.#......##
..#.#.....
###..##.##
...#....##
...#.#..##
.....#..#.
#..#......
#....####.
####..#..#

Tile 2089:
####...#.#
#.....#.##
.........#
.#...#....
...#..#.##
##.#......
##....#...
..#..#..##
...##..#..
#.##......

Tile 2719:
.##..#.#.#
#........#
.......#..
##...#....
..#.#.....
#..##....#
#.#..##.#.
#.........
#.........
..####.##.

Tile 1789:
..#.#.##.#
.....#.###
.....#...#
..#..#..##
..........
..##....##
##.#...#.#
.##...##.#
..####..#.
.#.##.##.#

Tile 1009:
.##.###.#.
#....##..#
###...#...
...#.#..##
#...#.#..#
#.##.....#
..#.#...#.
..#.......
.#.#......
.##.#.#.#.

Tile 2477:
...###.#..
..##......
#.#..#..#.
.#.#..####
#......#.#
.##..#....
##..##....
...#..#.#.
..........
.##..##.##

Tile 1013:
#.#..#####
#.#....#..
..#.#..###
#....##..#
.#.#.....#
#...#.##..
.#....##.#
..#......#
#.#..#.#..
#.##.#####

Tile 1451:
#..##..###
..###...##
.....#.#.#
#...#.....
...#.#....
#.#.......
.#..##...#
##..######
#.#....#.#
#.#..##.#.

Tile 1667:
..#.##..#.
#.#...#.##
.#.###...#
..##......
.........#
####..##.#
.........#
.....#..##
##...##..#
##..#.#.#.

Tile 1511:
.#..#..###
#..#..##.#
....#...##
....##...#
...##.....
#...#..##.
#.##..#.#.
..........
...#....##
###.#.##.#

Tile 2917:
.#####....
....#...#.
#..#.....#
#.....#.#.
...#.#.##.
#.##.....#
......####
#....#..#.
..#####..#
#.##.##.##

Tile 3877:
#....####.
#.........
#.....#.#.
.#.....#..
....#..#.#
##.##.##..
..#..#..#.
#..##....#
...#..#.#.
.....#..##

Tile 3319:
#.#.######
#.......#.
.#......##
.........#
#.....#.##
#..####...
#..###....
#.##...#.#
#.........
...##.#.##

Tile 1087:
##..####.#
#.##.....#
.........#
...#......
....#..##.
..#..#..##
.###..##.#
#..#.....#
.....##..#
#.#.######

Tile 1279:
.#####.#..
....#.....
.........#
##........
..#...#...
.#........
..###.##..
.....#....
#.##.#.#.#
#..#...#..

Tile 3257:
.......#..
...#..##..
#.#..#..##
.#..#.##.#
#.........
..#.......
#..######.
..#......#
....#.....
...###..#.

Tile 2579:
....####.#
....#..#.#
.#.......#
....#..#..
#.##...###
.#.#...###
##........
#.#.####..
#.#..##..#
####..##..

Tile 1091:
.#..###.##
.#....#..#
#.........
#...#..#.#
#.#......#
#.#.#.#..#
#.#.##..#.
.#....#.#.
#....#...#
##.##..###

Tile 1423:
.#....#...
.....#..#.
#.#....#.#
#.#####..#
#....#...#
.####.....
.#.###....
....#....#
#..#.##...
...#...#..

Tile 2531:
####.#..##
.#...##..#
........#.
.#....#.#.
...####..#
##........
#....#..##
...#....#.
#......#..
.........#

Tile 3767:
.#...#.##.
..#.#.#..#
#.#..#..#.
.......#.#
...###...#
#...###.##
#..#..#...
#....###..
#...####.#
.###.#####

Tile 2833:
#......##.
##.....##.
........#.
#...#..#..
#....#....
##......##
....####.#
......#.##
##....#...
#..#.##...

Tile 1291:
###.#..#..
#..##.#.#.
........##
#.......#.
...#......
......#.##
..........
.#.....###
#...##....
..##.####.

Tile 2399:
##.....#..
#..####..#
##...#..##
#....#.#.#
.##.#..#..
#.#......#
#.#..#....
#...#.#.##
..#....#..
...##..#..

Tile 1997:
.##.#..##.
..........
....#.....
......#.##
.#.##..#..
.........#
###..#..##
#..#.##...
..###.....
....####..

Tile 3119:
..##..#..#
#.#..#....
#.........
#...##.##.
##.....#.#
...#..#...
#..##....#
...#......
##..#.#...
.#..#.####

Tile 2417:
.#.#.##...
#..#.##...
....#...#.
.........#
......#.#.
.#...#....
###.....#.
....#.....
#..#.#....
####..#.##

Tile 3643:
.....##...
.#.#.###..
#..#......
##....#...
#.........
##.#....##
##.#......
#.#..####.
#.....#...
#..###...#

Tile 3491:
..##.##.#.
##.....#.#
#...##...#
####....#.
...#..#...
#........#
#...##...#
##.......#
###.....#.
#....#.##.

Tile 1103:
#.#...##.#
#.........
...#..##..
.....#...#
.........#
#...##..##
#..#..#..#
....#..#.#
#.....##..
####.#..#.

Tile 3001:
.#######..
#.##....##
..##......
##..#.....
....##.#..
#...#.....
#.....##..
#..#...###
......#...
.##.#..#..

Tile 3301:
..##.#..##
....#.....
#.#.....#.
...#......
#....#...#
#.#.....#.
#....#..#.
....##.###
........##
###.##..##

Tile 2411:
...###.###
.#.##.#...
.....#..#.
.....#....
.........#
.....#....
....#...#.
.....#....
##..#.....
##..#.#...

Tile 3769:
.#.#.#....
..#....##.
#....#...#
.#.......#
...#......
.#.....#.#
#.###..#.#
##.##....#
..#.##.#.#
.##.#.##.#

Tile 2791:
..##.#.#.#
#..#..#..#
...#......
.#.......#
....#.....
#.........
#.......#.
#....##..#
#.#.....#.
.###....#.

Tile 1381:
..##.##...
##........
#.........
..##.#.#.#
....#...##
##......#.
....#..#..
.....##.#.
#.....#.##
##..#....#

Tile 1181:
....#...#.
###..#..#.
##.......#
#....#.#.#
....#...#.
.##.#.#.#.
.#..#....#
.#.##.....
...##....#
...##..#.#

Tile 1999:
#..##.##.#
###..#....
#.........
.##.......
.#..##.#..
.###..#.#.
.......#.#
#.......#.
..#...##.#
..##..#...

Tile 2749:
.......#.#
.#....#..#
...#.#....
#.........
....###..#
.#...##...
....#.##..
#.#...#.#.
..#..##.##
#.###..#.#

Tile 2393:
#.###.....
#..#.###.#
.......#.#
#....###.#
#.....#...
...##..#.#
##........
##....#...
...#..###.
#...##..#.

Tile 2707:
..#..#####
###..##.#.
...#....##
..........
#..#..#...
#.#....##.
#.........
#.#......#
#.#.#....#
.#.##.....

Tile 1901:
##.#####..
#.#....#..
#..#.#.#..
....#.#.##
....#....#
..###.....
#.##.#.#.#
#.#...#..#
##.#..#..#
.##..#.#..

Tile 3821:
###..#####
#.....###.
#........#
#....#....
.#..#.#...
.#.#..#...
#.#..#...#
##........
.##.#.....
.....###..

Tile 2239:
#..#.##.#.
#...#.....
#.........
..#.#.#...
#....#.#..
.....#...#
#.#.#...#.
##...##...
..##.#.#.#
#.##.###..

Tile 2659:
#.####.###
#.......#.
...#.....#
....#..##.
#......#.#
.##...#...
##........
#...##.#.#
#.#...#..#
#..#####..

Tile 3517:
.#..###.#.
.##.#.#.#.
###.#...##
....#...#.
#.#.##....
..##..#...
.......#..
...#.....#
#.#......#
..####..##

Tile 2111:
######.##.
#....#...#
.....##..#
......#.##
.#.###...#
.##..#..##
##.......#
.#.....#.#
...#..#..#
.#.###.###

Tile 3109:
.#.#.##...
#...##...#
#......#.#
..#......#
#.#...#.##
#...#....#
......#..#
##.......#
#.#..##.##
#.#......#

Tile 3691:
...#...##.
##....##.#
..#...##..
...##..###
.#.#......
#.###..#..
...#......
#...#...##
.#........
.##.##....

Tile 1889:
#...#.#...
#.......#.
..##..#..#
#.....#..#
..#.#...##
#....#....
#...#.....
#...#..##.
#..#....##
#.....#.#.

Tile 1051:
.......##.
#.........
#..#..#..#
#......#.#
.....##...
..#...##..
#........#
#......#..
..#..#....
.....#.#.#

Tile 2633:
...#####..
#....#....
#.....#...
...##.....
..#.#.....
#.....##..
#..###..##
#..#.#...#
#..##.#.##
##...##.##

Tile 1949:
##.#..#...
.#....#...
#........#
#.#.#....#
#.......#.
#..###.##.
.#......##
.#........
...##.....
..#...#...

Tile 3169:
##..#.##..
..........
#......#..
#.........
..........
#...###..#
..#.####.#
#......#..
##..#....#
..##...##.

Tile 2027:
#..##...##
........#.
#.#......#
##.....#.#
##...##..#
..#.##.#..
#...#.#..#
#.#.......
..#.#..##.
...####.##

Tile 2851:
.####...##
#.#....#.#
..##......
...##.....
#...#...##
..##....#.
#....##.#.
.#..#...#.
#.##...#..
.##....#..

Tile 1213:
#.##.#####
#....#..##
#........#
#.##......
..#.#...##
..........
....#.#...
#....##...
#....#..#.
####.#.#.#

Tile 1847:
#..#..##.#
.........#
.........#
...#....#.
...#...#.#
#...#..#..
#...#..##.
###..#.#.#
#....####.
####.###.#

Tile 2843:
.###...##.
#..#....##
....#.#...
#.......##
#.#....#.#
......#..#
.....###..
##....#...
.......#..
##..##.#.#

Tile 1163:
..#.#..###
....##....
..#.#...#.
#.#.....##
.#........
##..#.....
#...#....#
...#..#..#
##...#.#.#
#.....#..#

Tile 3617:
.#.#.#.###
.....#.#.#
.........#
#........#
#.#.##.#.#
####.#..#.
.##...####
.......#..
###..#...#
..####.#.#

Tile 2963:
#.......#.
.....#...#
.##......#
#.#....###
..##...#..
#.###..#.#
....#....#
..#.#...##
#..###....
#..#..#.#.

Tile 1709:
#...#.##..
.#....####
.....#.#.#
#........#
#..###....
#......###
###.....#.
....#..#..
.##.#....#
.#.#...#..

Tile 1583:
.#.##.#.#.
#.....#..#
.....##...
##....#..#
##.#.....#
.#.......#
#.##.#..##
##..#..###
...#....#.
..#.###.#.

Tile 3881:
####.#.#.#
.....#.#.#
#......#.#
...#......
.#.....#..
....#.#.##
..........
##.....#..
#...##...#
..#.###..#""".split('\n\n')